# Here we prepair the human analys for the results of GPT for the input with textual representation and annotations

In [1]:
from support_functions.load_json_chatgpt_results import load_chat_gpt_results
from support_functions.excel_helper import prepaire_excel_data
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.data_transformations import convert_annotation_dataset

import json
import pandas as pd
import copy
from enum import Enum
from ordered_set import OrderedSet

In [2]:
class RedundancyType(Enum):
    MAIN_PART_REDUNDANCY = "mainPart"
    BENEFIT_REDUNDANCY = "benefit"
    MAIN_PART_BENEFIT_REDUNDANCY = "mainPart-benefit"

In [3]:
def clean_convertable(filtered_rows):
        convertable = json.loads(filtered_rows.to_json(orient="records"))[0]
        del convertable["Project Identifier"]
        del convertable["User Story 1 Text with Markers"]
        del convertable["User Story 2 Text with Markers"]
        del convertable["Total Redundancy Clause"]
        del convertable["USID 1"]
        del convertable["USID 2"]
        return convertable

In [4]:
def calc_repair_avg(json_data: list[dict]) -> int:
    count: int = 0
    i: int = 0
    temp: int = None
    for usid_redundancy_key, item in json_data.items():
        model = usid_redundancy_key
        for sub_key, sub_item in item.items():
            data_set = sub_key
            for _ in sub_item:
                temp = int(_["repairRuns"])
                if temp > 0:
                    count += temp
                i += 1            
    return round(count / i, 2)

In [5]:
def calc_total_repair(json_data: list[dict]) -> int:
    count: int = 0
    temp: int = None
    for usid_redundancy_key, item in json_data.items():
        model = usid_redundancy_key
        for sub_key, sub_item in item.items():
            data_set = sub_key
            for _ in sub_item:
                temp = int(_["repairRuns"])
                if temp > 0:
                    count += temp
    return count

## Getting Graph-Transformation Data

In [6]:
excel_data_graph_approach = prepaire_excel_data("Framework")
display(excel_data_graph_approach.head(3))

,Identifier,Project Identifier,USID 1,User Story 1 Text with Markers,USID 2,User Story 2 Text with Markers,Total Redundancy Clause,Main Part Redundancy Clause,Benefit Part Redundancy Clause,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,g19user_story_1021_AND_user_story_1036,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1036,"As an OlderPerson, I want to #use# #ALFRED# to...",2,2,0,True,False,False,False
2,g19user_story_1021_AND_user_story_1056,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1056,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False
3,g19user_story_1021_AND_user_story_1057,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1057,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False


## Graph-Transformation Results and GPT Text Results

In [7]:
directories_to_ignore_text:list = ["text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125",
                              "strict_redundancy_text-redundancy-model-gpt-4o-mini-2024-07-18", "strict-annoations-redundancy-model-gpt-4o-mini-2024-07-18"]
gpt_results_text: dict[dict[str, list[dict]]] = load_chat_gpt_results(directories_to_ignore_text)
gpt_result_keys_text: list[str] = list(gpt_results_text.keys())
gpt_result_json_keys_text = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in gpt_results_text.items()
}

### Basic analyses

In [8]:
redundant_main_part_text: int = 0
redundant_benefit_text: int = 0
no_redundancies_main_part_text: int = 0
no_redundancies_benefit_text: int = 0

In [9]:
for usid_redundancy_key, item in gpt_results_text.items():
    model = usid_redundancy_key
    for sub_key, sub_item in item.items():
        data_set = sub_key
        for _ in sub_item:
            if bool(_["mainPartRedundancies"]["mainPartRedundancy"]):
                redundant_main_part_text += 1
            else:
                no_redundancies_main_part_text += 1
            if bool(_["benefitRedundancies"]["benefitRedundancy"]):
                redundant_benefit_text += 1
            else:
                no_redundancies_benefit_text += 1
        print(f"{model}, {data_set}, Redundancies in Main Part: {redundant_main_part_text}, No Reduncancies in Main Part: {no_redundancies_main_part_text}, Redundancies in Benefit: {redundant_benefit_text}, No Redundancies in Benefit: {no_redundancies_benefit_text}")
        redundant_main_part_text = no_redundancies_main_part_text = redundant_benefit_text = no_redundancies_benefit_text = 0

text-redundancy-model-gpt-4o-mini-2024-07-18, 00_g19.json, Redundancies in Main Part: 1788, No Reduncancies in Main Part: 7410, Redundancies in Benefit: 46, No Redundancies in Benefit: 9152
text-redundancy-model-gpt-4o-mini-2024-07-18, 00_g22.json, Redundancies in Main Part: 541, No Reduncancies in Main Part: 2860, Redundancies in Benefit: 124, No Redundancies in Benefit: 3277


In [10]:
print(calc_repair_avg(gpt_results_text))
print(calc_total_repair(gpt_results_text))

0.03
384


### Get all Redundant US with text (add to json)

In [11]:
not_processed_datasets_text: dict[str, list] = loading()
datasets_text, ignored_items = convert_annotation_dataset(not_processed_datasets_text)

g19_text = datasets_text["g19"]
g22_text = datasets_text["g22"]
print(g22_text)

[{'PID': '#G22#', 'USID': '1225', 'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.', 'Main Part': 'As a PI, I want to properly record all metadata', 'Benefit': 'I can ensure proper running of the project in case of staff changes', 'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []}, 'Targets': {'Main Part': [['properly record', 'all metadata']], 'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]}, 'Contains': {'Main Part': [], 'Benefit': []}}, {'PID': '#G22#', 'USID': '1226', 'Text': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project, so that it remains accessible during all stages of the data lifecycle.', 'Main Part': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project', 'Benefit': 'it remains accessible during all stages of the data lifecycle', 'Tr

In [12]:
already_processed_text: bool = False

In [13]:
cached_entries_just_text: list[dict] = []
if not already_processed_text:
    already_processed_text = True
    entry1: dict = None
    entry2: dict = None
    entries_to_save: list[dict] = []
    both_entries: int = 0  # Initialize both_entries outside the loop
    copy_item = None
    for usid_redundancy_key, item in gpt_results_text.items():
        model = usid_redundancy_key
        copy_item = copy.deepcopy(item)
        for sub_key, sub_item in copy_item.items():
            data_set = sub_key
            entries_to_save.append(sub_item)
            sub_item[:] = [
                _ for _ in sub_item 
                if bool(_["mainPartRedundancies"]["mainPartRedundancy"]) 
                or bool(_["benefitRedundancies"]["benefitRedundancy"])
            ]
            
            for _ in sub_item:  
                usid1, usid2 = _["relatedStories"]
                _["1_userStoryText"] = ""
                _["2_userStoryText"] = ""
                _["1_userStoryMainPart"] = ""
                _["2_userStoryMainPart"] = ""
                _["1_userStoryBenefit"] = ""
                _["2_userStoryBenefit"] = ""
                
                for entry in g19_text + g22_text:
                    if int(entry["USID"]) == usid1:
                        entry1 = entry
                        _["1_userStoryText"] = entry["Text"]
                        _["1_userStoryMainPart"] = entry["Main Part"]
                        _["1_userStoryBenefit"] = entry["Benefit"]
                        both_entries += 1
                    if int(entry["USID"]) == usid2:
                        entry2 = entry
                        _["2_userStoryText"] = entry["Text"]
                        _["2_userStoryMainPart"] = entry["Main Part"]
                        _["2_userStoryBenefit"] = entry["Benefit"]
                        both_entries += 1
                    if both_entries == 2:
                        break                
                both_entries = 0
                
    file1 = 'for_debugging_proposes_g19_enriched_text-redundancy-model-gpt-4o-mini-2024-07-18.json'
    file2 = 'for_debugging_proposes_g22_enriched_text-redundancy-model-gpt-4o-mini-2024-07-18.json'         
    with open(file1, 'w') as f1:
        json.dump(entries_to_save[0], f1, indent=4)
    with open(file2, 'w') as f2:
        json.dump(entries_to_save[1], f2, indent=4)
        
    cached_entries_just_text = copy.deepcopy(entries_to_save)

### Getting overlaps between graph transformation approach and llm (just text)

In [14]:
overlaps_graph_llm_text: dict[str, dict[str, list[dict[str, dict]]]] = {} # first key is the dataset, secondary key is the pair numbers, thirdly a list of the two json entries is stored
overlaps_graph_llm_output_text: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type
non_overlaps_graph_llm_text: dict[str, dict[str, list[dict[str, dict]]]] = {} # first key is the dataset, the secondary key is the pair numbers, the value is the pair numbers with redundancy type
non_overlaps_graph_llm_output_text: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp_list: list[dict[str, dict]] = []
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_text.values():
        for dataset_key, array in dataset.items():
            if dataset_key not in overlaps_graph_llm_text:
                overlaps_graph_llm_text[dataset_key] = []
            if dataset_key not in non_overlaps_graph_llm_text:
                non_overlaps_graph_llm_text[dataset_key] = []
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                
                entries_in_filtered_row = len(filtered_rows)
                if  entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                if entries_in_filtered_row == 1:
                    convertable_gt_system_output = clean_convertable(filtered_rows)
                    
                if (bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]) and 
                    not bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"])):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.MAIN_PART_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1 and 
                         (bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                          or bool(filtered_rows.iloc[0]["Main Part Full"]))):
                        temp_list = overlaps_graph_llm_text[dataset_key]
                        temp_list.append([convertable_gt_system_output, llm_detection_values])
                        overlaps_graph_llm_output_text.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_text[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[{}, llm_detection_values]})
                        non_overlaps_graph_llm_output_text.add(usid_redundancy_key)
                if (bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]) 
                    and not bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"])):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.BENEFIT_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1
                            and (bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                            or bool(filtered_rows.iloc[0]["Benefit Full"]))):                              
                        temp_list = overlaps_graph_llm_text[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[convertable_gt_system_output, llm_detection_values]})
                        overlaps_graph_llm_output_text.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_text[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[{}, llm_detection_values]})
                        non_overlaps_graph_llm_output_text.add(usid_redundancy_key)
                if (bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]) 
                    and bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"])):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1
                            and (bool(filtered_rows.iloc[0]["Main Part Partial"]) or bool(filtered_rows.iloc[0]["Main Part Full"])) 
                            and (bool(filtered_rows.iloc[0]["Benefit Partial"]) or bool(filtered_rows.iloc[0]["Benefit Full"]))):  
                        temp_list = overlaps_graph_llm_text[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[convertable_gt_system_output, llm_detection_values]})
                        overlaps_graph_llm_output_text.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_text[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[{}, llm_detection_values]})
                        non_overlaps_graph_llm_output_text.add(usid_redundancy_key)
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None

In [15]:
len(overlaps_graph_llm_output_text)

468

In [16]:
for item in overlaps_graph_llm_output_text:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value  and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1021,1036,,-
1021,1056,,-
1021,1057,,-
1021,1059,,-
1021,1060,,-
1021,1061,,-
1021,1062,,-
1021,1063,,-
1021,1064,,-
1021,1065,,-
1021,1066,,-
1021,1073,,-
1021,1074,,-
1021,1081,,-
1021,1082,,-
1021,1087,,-
1021,1088,,-
1021,1089,,-
1021,1090,,-
1021,1091,,-
1021,1092,,-
1021,1093,,-
1021,1095,,-
1026,1027,,-
1026,1068,,-
1026,1075,,-
1026,1076,,-
1026,1077,,-
1026,1078,,-
1026,1098,,-
1026,1107,,-
1026,1108,,-
1026,1110,,-
1026,1115,,-
1026,1116,,-
1026,1121,,-
1026,1126,,-
1026,1128,,-
1026,1129,,-
1026,1134,,-
1027,1068,,-
1027,1075,,-
1027,1077,,-
1027,1078,,-
1027,1098,,-
1027,1107,,-
1027,1108,,-
1027,1115,,-
1027,1116,,-
1027,1121,,-
1027,1126,,-
1027,1128,,-
1027,1129,,-
1027,1134,,-
1029,1030,,-
1029,1031,,-
1029,1038,,-
1029,1039,,-
1029,1040,,-
1029,1041,,-
1029,1042,,-
1029,1044,,-
1029,1045,,-
1029,1049,,-
1030,1031,,-
1030,1038,,-
1030,1039,,-
1030,1040,,-
1030,1041,,-
1030,1042,,-
1030,1044,,-
1030,1045,,-
1030,1049,,-
1031,1038,,-
1031,1039,,-
1031,1040,,-
1031,1041,,-

In [17]:
len(non_overlaps_graph_llm_output_text)

1937

In [18]:
strict_llm_results_text: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:
    mp_overlaps:int = 0
    mp_non_overlaps:int = 0
    mp_negatives:int = 0
    
    benefit_overlaps:int = 0
    benefit_non_overlaps:int = 0
    benefit_negatives:int = 0
    
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp:str = None
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_text.values():
        for dataset_key, array in dataset.items():
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                
                entries_in_filtered_row = len(filtered_rows)
                if entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                temp = ""
                try:
                    if (not bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Main Part Full"])):
                        if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                            mp_non_overlaps += 1
                            temp += "main part,fp"
                        else:
                            mp_negatives += 1
                            temp += "main part,tn"
                    else: 
                        if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                            mp_overlaps += 1
                            temp += "main part,tp"
                        else:
                            mp_non_overlaps += 1
                            temp += "main part,fn"
                    
                    temp += ","
                    
                    if (not bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Benefit Full"])):
                        if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                            benefit_non_overlaps += 1
                            temp += "benefit,fp"
                        else:
                            benefit_negatives += 1
                            temp += "benefit,tn"
                    else: 
                        if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                            benefit_overlaps += 1
                            temp += "benefit,tp"
                        else:
                            benefit_non_overlaps += 1
                            temp += "benefit,fn"
                except:
                    if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                        mp_non_overlaps += 1
                        temp += "main part,fp"
                    else:
                        mp_negatives += 1
                        temp += "main part,tn"
                        
                    temp += ","
                    
                    if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                        benefit_non_overlaps += 1
                        temp += "benefit,fp"
                    else:
                        benefit_negatives += 1
                        temp += "benefit,tn"
                        
                if len(temp) > 0:
                    temp = f"{usid1},{usid2},{temp}"
                    strict_llm_results_text.add(temp)                                
                
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None
    print(f"Main Part: Overlaps: {mp_overlaps}, Non-Overlapping: {mp_non_overlaps}, Negatives: {mp_negatives}")
    print(f"Benefit: Overlaps: {benefit_overlaps}, Non-Overlapping: {benefit_non_overlaps}, Negatives: {benefit_negatives}")

Main Part: TP: 478, FP: 1851, FN: 102, TN: 10168
Benefit: TP: 23, FP: 147, FN: 12, TN: 12417


## Graph-Transformation-Results and GPT-Annotation Results

In [19]:
directories_to_ignore_annotations:list[str] = ["text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "text-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125",
                              "strict_redundancy_text-redundancy-model-gpt-4o-mini-2024-07-18", "strict-annoations-redundancy-model-gpt-4o-mini-2024-07-18"]
gpt_results_annotations: dict[dict[str, list[dict]]] = load_chat_gpt_results(directories_to_ignore_annotations)
gpt_result_keys_annotations: list[str] = list(gpt_results_annotations.keys())
gpt_result_json_keys_annotations = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in gpt_results_annotations.items()
}

### Basic analyses

In [20]:
redundant_main_part_annotations: int = 0
redundant_benefit_annotations: int = 0
no_redundancies_main_part_annotations: int = 0
no_redundancies_benefit_annotations: int = 0

In [21]:
for usid_redundancy_key, item in gpt_results_annotations.items():
    model = usid_redundancy_key
    for sub_key, sub_item in item.items():
        data_set = sub_key
        for _ in sub_item:
            if (bool(_["mainPartRedundancies"]["fullRedundancy"]) 
                    or bool(_["mainPartRedundancies"]["partialRedundancy"])):
                redundant_main_part_annotations += 1
            else:
                no_redundancies_main_part_annotations += 1
            if (bool(_["benefitRedundancies"]["fullRedundancy"]) 
                    or bool(_["benefitRedundancies"]["partialRedundancy"])):
                redundant_benefit_annotations += 1
            else:
                no_redundancies_benefit_annotations += 1
        print(f"{model}, {data_set}, Redundancies in Main Part: {redundant_main_part_annotations}, No Reduncancies in Main Part: {no_redundancies_main_part_annotations}, Redundancies in Benefit: {redundant_benefit_annotations}, No Redundancies in Benefit: {no_redundancies_benefit_annotations}")
        redundant_main_part_annotations = no_redundancies_main_part_annotations = redundant_benefit_annotations = no_redundancies_benefit_annotations = 0

annoations-redundancy-model-gpt-4o-mini-2024-07-18, 00_g19.json, Redundancies in Main Part: 1585, No Reduncancies in Main Part: 7699, Redundancies in Benefit: 292, No Redundancies in Benefit: 8992
annoations-redundancy-model-gpt-4o-mini-2024-07-18, 00_g22.json, Redundancies in Main Part: 328, No Reduncancies in Main Part: 3056, Redundancies in Benefit: 192, No Redundancies in Benefit: 3192


In [22]:
print(calc_repair_avg(gpt_results_annotations))
print(calc_total_repair(gpt_results_annotations))

0.04
472


### Get all Redundant US with text (add to json)

In [23]:
not_processed_datasets_annotations: dict[str, list] = loading()
datasets_annotations, ignored_items = convert_annotation_dataset(not_processed_datasets_annotations)

g19_annotations = datasets_annotations["g19"]
g22_annotations = datasets_annotations["g22"]
print(g22_annotations)

[{'PID': '#G22#', 'USID': '1225', 'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.', 'Main Part': 'As a PI, I want to properly record all metadata', 'Benefit': 'I can ensure proper running of the project in case of staff changes', 'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []}, 'Targets': {'Main Part': [['properly record', 'all metadata']], 'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]}, 'Contains': {'Main Part': [], 'Benefit': []}}, {'PID': '#G22#', 'USID': '1226', 'Text': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project, so that it remains accessible during all stages of the data lifecycle.', 'Main Part': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project', 'Benefit': 'it remains accessible during all stages of the data lifecycle', 'Tr

In [24]:
already_processed_annotations: bool = False

In [25]:
cached_entries_just_annotations: list[dict] = []
if not already_processed_annotations:
    already_processed_annotations = True
    entry1: dict = None
    entry2: dict = None
    entries_to_save: list[dict] = []
    copy_item: list[dict] = None
    both_entries: int = 0  # Initialize both_entries outside the loop
    for usid_redundancy_key, item in gpt_results_annotations.items():
        model = usid_redundancy_key
        copy_item = copy.deepcopy(item)
        for sub_key, sub_item in copy_item.items():
            data_set = sub_key
            entries_to_save.append(sub_item)
            sub_item[:] = [
                _ for _ in sub_item 
                if bool(_["mainPartRedundancies"]["fullRedundancy"])
                or bool(_["mainPartRedundancies"]["partialRedundancy"])
                or bool(_["benefitRedundancies"]["fullRedundancy"])
                or bool(_["benefitRedundancies"]["partialRedundancy"])
            ]
            
            for _ in sub_item:
                usid1, usid2 = _["relatedStories"]
                _["1_userStoryText"] = ""
                _["2_userStoryText"] = ""
                _["1_userStoryMainPart"] = ""
                _["2_userStoryMainPart"] = ""
                _["1_userStoryBenefit"] = ""
                _["2_userStoryBenefit"] = ""
                
                for entry in g19_annotations + g22_annotations:
                    if int(entry["USID"]) == usid1:
                        entry1 = entry
                        _["1_userStoryText"] = entry["Text"]
                        _["1_userStoryMainPart"] = entry["Main Part"]
                        _["1_userStoryBenefit"] = entry["Benefit"]
                        both_entries += 1
                    if int(entry["USID"]) == usid2:
                        entry2 = entry
                        _["2_userStoryText"] = entry["Text"]
                        _["2_userStoryMainPart"] = entry["Main Part"]
                        _["2_userStoryBenefit"] = entry["Benefit"]
                        both_entries += 1
                    if both_entries == 2:
                        break                
                both_entries = 0
    
    file1 = 'for_debugging_proposes_g19_enriched_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json'
    file2 = 'for_debugging_proposes_g22_enriched_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json'              
    with open(file1, 'w') as f1:
        json.dump(entries_to_save[0], f1, indent=4)
    with open(file2, 'w') as f2:
        json.dump(entries_to_save[1], f2, indent=4)
        
    cached_entries_just_annotations = copy.deepcopy(entries_to_save)

## Getting the non-/overlaps between graph transformation approach and llm

In [26]:
overlaps_graph_llm_annotations: dict[str, dict[str, list[dict[str, dict]]]] = {} # first key is the dataset, secondary key is the pair numbers, thirdly a list of the two json entries is stored
overlaps_graph_llm_output_annotations: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type
non_overlaps_graph_llm_annotations: dict[str, dict[str, list[dict[str, dict]]]] = {} # first key is the dataset, the secondary key is the pair numbers, the value is the pair numbers with redundancy type
non_overlaps_graph_llm_output_annotations: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:        
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp_list: list[dict[str, dict]] = []
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_annotations.values():
        for dataset_key, array in dataset.items():
            if dataset_key not in overlaps_graph_llm_annotations:
                overlaps_graph_llm_annotations[dataset_key] = []
            if dataset_key not in non_overlaps_graph_llm_annotations:
                non_overlaps_graph_llm_annotations[dataset_key] = []
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                    
                entries_in_filtered_row = len(filtered_rows)
                if  entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                if entries_in_filtered_row == 1:
                    convertable_gt_system_output = clean_convertable(filtered_rows)
                    
                if ((
                            bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])
                        ) 
                        and not (
                            bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])
                        )):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.MAIN_PART_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1 and 
                         (bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                          or bool(filtered_rows.iloc[0]["Main Part Full"]))):
                        temp_list = overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append([convertable_gt_system_output, llm_detection_values])
                        overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append([{}, llm_detection_values])
                        non_overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                if (not (
                            bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])
                        ) 
                        and (
                            bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])
                        )):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.BENEFIT_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1
                            and (bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                            or bool(filtered_rows.iloc[0]["Benefit Full"]))):                              
                        temp_list = overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[convertable_gt_system_output, llm_detection_values]})
                        overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[{}, llm_detection_values]})
                        non_overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                if ((
                            bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])
                        ) 
                        and (
                            bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                            or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])
                        )):
                    usid_redundancy_key = f"{usid1}-{usid2}-{RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value}"
                    if  (entries_in_filtered_row == 1
                            and (bool(filtered_rows.iloc[0]["Main Part Partial"]) or bool(filtered_rows.iloc[0]["Main Part Full"])) 
                            and (bool(filtered_rows.iloc[0]["Benefit Partial"]) or bool(filtered_rows.iloc[0]["Benefit Full"]))):  
                        temp_list = overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[convertable_gt_system_output, llm_detection_values]})
                        overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                    else:
                        temp_list = non_overlaps_graph_llm_annotations[dataset_key]
                        temp_list.append({"usid_redundcy_key":usid_redundancy_key, "value":[{}, llm_detection_values]})
                        non_overlaps_graph_llm_output_annotations.add(usid_redundancy_key)
                    
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None

In [27]:
len(overlaps_graph_llm_output_annotations)

393

In [28]:
for item in overlaps_graph_llm_output_annotations:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1021,1036,,-
1021,1056,,-
1021,1057,,-
1021,1059,,-
1021,1060,,-
1021,1061,,-
1021,1062,,-
1021,1064,,-
1021,1066,,-
1021,1073,,-
1021,1074,,-
1021,1081,,-
1021,1082,,-
1021,1087,,-
1021,1088,,-
1021,1089,,-
1021,1090,,-
1021,1091,,-
1021,1092,,-
1021,1093,,-
1021,1095,,-
1026,1027,,-
1026,1068,,-
1026,1075,,-
1026,1076,,-
1026,1077,,-
1026,1078,,-
1026,1098,,-
1026,1107,,-
1026,1108,,-
1026,1109,,-
1026,1110,,-
1026,1113,,-
1026,1115,,-
1026,1116,,-
1026,1121,,-
1026,1122,,-
1026,1126,,-
1026,1128,,-
1026,1129,,-
1026,1134,,-
1027,1068,,-
1027,1075,,-
1027,1076,,-
1027,1077,,-
1027,1078,,-
1027,1098,,-
1027,1107,,-
1027,1108,,-
1027,1109,,-
1027,1110,,-
1027,1113,,-
1027,1115,,-
1027,1116,,-
1027,1121,,-
1027,1122,,-
1027,1126,,-
1027,1128,,-
1027,1129,,-
1027,1134,,-
1029,1030,,-
1029,1031,,-
1029,1038,,-
1029,1039,,-
1029,1040,,-
1029,1041,,-
1029,1042,,-
1029,1044,,-
1029,1045,,-
1029,1049,,-
1030,1031,,-
1030,1042,,-
1030,1044,,-
1031,1038,,-
1031,1039,,-
1031,1040,,-
1031,1041,,-

In [29]:
len(non_overlaps_graph_llm_output_annotations)

1711

In [30]:
strict_llm_results_annotations: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:
    mp_overlaps:int = 0
    mp_non_overlaps:int = 0
    mp_negatives:int = 0
    
    benefit_overlaps:int = 0
    benefit_non_overlaps:int = 0
    benefit_negatives:int = 0
    
    
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp_list: list[dict[str, dict]] = []
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_annotations.values():
        for dataset_key, array in dataset.items():
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                    
                entries_in_filtered_row = len(filtered_rows)
                if  entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                temp = ""
                try:
                    if (not bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Main Part Full"])):
                        if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            mp_non_overlaps += 1
                            temp += "main part,fp"
                        else:
                            mp_negatives += 1
                            temp += "main part,tn"
                    else: 
                        if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            mp_overlaps += 1
                            temp += "main part,tp"
                        else:
                            mp_non_overlaps += 1
                            temp += "main part,fn"
                    
                    temp += ","
                    
                    
                    if (not bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Benefit Full"])):
                        if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                            benefit_non_overlaps += 1
                            temp += "benefit,fp"
                        else:
                            benefit_negatives += 1
                            temp += "benefit,tn"
                    else: 
                        if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                            benefit_overlaps += 1
                            temp += "benefit,tp"
                        else:
                            benefit_negatives += 1
                            temp += "benefit,fn"
                except:
                    if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            mp_non_overlaps += 1
                            temp += "main part,fp"
                    else:
                        mp_negatives += 1
                        temp += "main part,tn"
                    temp += ","
                    if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                        benefit_non_overlaps += 1
                        temp += "benefit,fp"
                    else:
                        benefit_negatives += 1
                        temp += "benefit,tn"
                        
                if len(temp) > 0:
                    temp = f"{usid1},{usid2},{temp}"
                    strict_llm_results_annotations.add(temp)                                
                
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None
    print(f"Main Part: Overlaps: {mp_overlaps}, Non-Overlaps: {mp_non_overlaps}, FN: {fn_main_part}, TN: {tn_main_part}")
    print(f"Benefit: Overlaps: {benefit_overlaps}, Non-Overlaps: {benefit_non_overlaps}, FN: {fn_benefit}, TN: {tn_benefit}")

Main Part: TP: 565, FP: 1348, FN: 8, TN: 10747
Benefit: TP: 35, FP: 449, FN: 0, TN: 12184


## Getting the non-/overlaps between the two llm approaches and llm and graph transformation approach

In [31]:
getting_redundancies_which_are_either_text_and_graph_or_annoations_and_graph: OrderedSet = None
getting_all_text_graph_corrects_wich_are_in_annotations_graph_corrects: OrderedSet = None
getting_text_graph_corrects_which_are_not_in_annotation_graph_corrects: OrderedSet = None
getting_all_annotations_graph_corrects_wich_are_in_text_graph_corrects: OrderedSet = None
getting_annotation_graph_corrects_which_are_not_in_text_graph_corrects: OrderedSet = None
all_which_are_in_the_other_and_determined_incorrect_by_graph: OrderedSet = None
all_which_are_not_in_the_other_and_determined_incorrect_by_graph: OrderedSet = None

In [32]:
if True:
    temp1: OrderedSet = None
    temp2: OrderedSet = None
    
    # Getting the differences between text + graph and annotations + graph
    temp1 = copy.deepcopy(overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(overlaps_graph_llm_output_annotations)
    temp1.symmetric_difference_update(temp2)
    getting_redundancies_which_are_either_text_and_graph_or_annoations_and_graph = temp1
    # Getting all annotations + graph that are not in text + graph
    temp1 = copy.deepcopy(overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(overlaps_graph_llm_output_annotations)
    temp1.intersection_update(temp2)
    getting_all_text_graph_corrects_wich_are_in_annotations_graph_corrects = temp1
    # Getting all annotations + graph that are not in text + graph
    temp1 = copy.deepcopy(overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(overlaps_graph_llm_output_annotations)
    getting_text_graph_corrects_which_are_not_in_annotation_graph_corrects = temp1.difference(temp2)
    # Getting all text + graph that are not in annotations + graph
    temp1 = copy.deepcopy(overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(overlaps_graph_llm_output_annotations)
    temp2.intersection_update(temp1)
    getting_all_annotations_graph_corrects_wich_are_in_text_graph_corrects = temp2
    # Getting all text + graph that are not in annotations + graph
    temp1 = copy.deepcopy(overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(overlaps_graph_llm_output_annotations)
    getting_annotation_graph_corrects_which_are_not_in_text_graph_corrects = temp2.difference(overlaps_graph_llm_output_annotations)
    # Getting all which are not matching in text + graph and are not in annotations + graph
    temp1 = copy.deepcopy(non_overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(non_overlaps_graph_llm_output_annotations)
    temp1.difference_update(temp2)
    getting_all_non_overlaps_graph_text_which_is_not_in_graph_annotations = temp1
    # Getting all which are not matching in annotations + graph and are not in text + graph
    temp1 = copy.deepcopy(non_overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(non_overlaps_graph_llm_output_annotations)
    temp2.difference_update(temp1)
    getting_all_non_overlaps_graph_annotations_which_is_not_in_graph_text = temp2
    # Getting all which are in the other and determined not correct by graph
    temp1 = copy.deepcopy(non_overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(non_overlaps_graph_llm_output_annotations)
    temp1.intersection_update(temp2)
    all_which_are_in_the_other_and_determined_incorrect_by_graph = temp1
    # Getting all which are not in the other and determined correct by graph
    temp1 = copy.deepcopy(non_overlaps_graph_llm_output_text)
    temp2 = copy.deepcopy(non_overlaps_graph_llm_output_annotations)
    temp1.symmetric_difference_update(temp2)
    all_which_are_not_in_the_other_and_determined_incorrect_by_graph = temp1
    

In [33]:
len(all_which_are_in_the_other_and_determined_incorrect_by_graph)

953

In [34]:
len(all_which_are_not_in_the_other_and_determined_incorrect_by_graph)

1742

In [35]:
already_processed: bool = False

In [36]:
if True:
    cached_entries_non_overlaps: dict[list[dict]]

    if not already_processed:
        with open('for_debugging_proposes_g19_enriched_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json', 'r') as file:
            data_g19_annotations = json.load(file)
        with open('for_debugging_proposes_g19_enriched_text-redundancy-model-gpt-4o-mini-2024-07-18.json', 'r') as file:
            data_g19_text = json.load(file)
        with open('for_debugging_proposes_g22_enriched_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json', 'r') as file:
            data_g22_annotations = json.load(file)
        with open('for_debugging_proposes_g22_enriched_text-redundancy-model-gpt-4o-mini-2024-07-18.json', 'r') as file:
            data_g22_text = json.load(file)
        cached_entries_non_overlaps = {"text_g19": data_g19_text, "text_g22": data_g22_text, "annotations_g19": data_g19_annotations, "annotations_g22": data_g22_annotations}
        
        
    if not already_processed:
        # already_processed_text = True
        entry1: dict = None
        entry2: dict = None
        usid1: int = 0
        usid2: int = 0
        entries_to_save: list[dict] = []
        both_entries: int = 0  # Initialize both_entries outside the loop
        copy_item_text: dict[str, list[dict]] = {"g19": [], "g22": []}
        copy_item_annotations: dict[str, list[dict]] = {"g19": [], "g22": []}
        temp_container_copy: any = None
        temp_item_copy: any = None
        to_remove: list[dict] = []
        
        temp_container_copy = cached_entries_non_overlaps["text_g19"]
        for item in temp_container_copy:
            usid1, usid2 = item["relatedStories"]
            for us_pair in list(all_which_are_in_the_other_and_determined_incorrect_by_graph) + list(overlaps_graph_llm_output_text):
                if str(usid1) in us_pair and str(usid2) in us_pair:
                    to_remove.append(item)
                    break
        
        for remove_item in to_remove:
            temp_container_copy.remove(remove_item)
        to_remove.clear()
        copy_item_text["g19"] = copy.deepcopy(temp_container_copy)
            
        temp_container_copy = cached_entries_non_overlaps["text_g22"]
        for item in temp_container_copy:
            usid1, usid2 = item["relatedStories"]
            for us_pair in list(all_which_are_in_the_other_and_determined_incorrect_by_graph) + list(overlaps_graph_llm_output_text):
                if str(usid1) in us_pair and str(usid2) in us_pair:
                    if item not in to_remove:
                        to_remove.append(item)
                    break
        
        for remove_item in to_remove:
            temp_container_copy.remove(remove_item)
        to_remove.clear()
        copy_item_text["g22"] = copy.deepcopy(temp_container_copy)
        
        temp_container_copy = cached_entries_non_overlaps["annotations_g19"]
        for item in temp_container_copy:
            usid1, usid2 = item["relatedStories"]
            for us_pair in list(all_which_are_in_the_other_and_determined_incorrect_by_graph) + list(overlaps_graph_llm_output_annotations):
                if str(usid1) in us_pair and str(usid2) in us_pair:
                    if item not in to_remove:
                        to_remove.append(item)
                    break
        
        for remove_item in to_remove:
            temp_container_copy.remove(remove_item)
        to_remove.clear()
        copy_item_annotations["g19"] = copy.deepcopy(temp_container_copy)
        
        temp_container_copy = cached_entries_non_overlaps["annotations_g22"]
        for item in temp_container_copy:
            usid1, usid2 = item["relatedStories"]
            for us_pair in list(all_which_are_in_the_other_and_determined_incorrect_by_graph) + list(overlaps_graph_llm_output_annotations):
                if str(usid1) in us_pair and str(usid2) in us_pair:
                    if item not in to_remove:
                        to_remove.append(item)
                    break
        
        for remove_item in to_remove:
            temp_container_copy.remove(remove_item)
        to_remove.clear()
        copy_item_annotations["g22"] = copy.deepcopy(temp_container_copy)

    file0 = 'g19_enriched_differences_not_in_other_sets_text-redundancy-model-gpt-4o-mini-2024-07-18.json'
    file1 = 'g22_enriched_differences_not_in_other_sets_text-redundancy-model-gpt-4o-mini-2024-07-18.json'         
    with open(file0, 'w') as f0:
        json.dump(copy_item_text["g19"], f0, indent=4)
    with open(file1, 'w') as f1:
        json.dump(copy_item_text["g22"], f1, indent=4)
        
    file2 = 'g19_enriched_differences_not_in_other_sets_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json'
    file3 = 'g22_enriched_differences_not_in_other_sets_annotations-redundancy-model-gpt-4o-mini-2024-07-18.json'              
    with open(file2, 'w') as f2:
        json.dump(copy_item_annotations["g19"], f2, indent=4)
    with open(file3, 'w') as f3:
        json.dump(copy_item_annotations["g22"], f3, indent=4)
        
    entry1 = None
    entry2 = None
    usid1 = None
    usid2 = None
    entries_to_save = None
    both_entries = None
    copy_item_text = None
    copy_item_annotations = None
    temp_container_copy = None
    temp_item_copy = None
    to_remove = None

### Templateing the analyse excel formats

In [37]:
benefit: str = "-"
main_part: str = "-"
split: list[str] = None

In [38]:
# G19+G22 Text
for item in getting_all_non_overlaps_graph_text_which_is_not_in_graph_annotations:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}") #


1018,1019,,-
1018,1023,,-
1018,1026,,-
1018,1030,,-
1018,1036,,-
1018,1061,,-
1018,1062,,-
1018,1065,,-
1018,1078,,-
1018,1087,,-
1018,1101,,-
1018,1105,,-
1019,1020,,-
1019,1021,,-
1019,1026,,-
1019,1029,,-
1019,1030,,-
1019,1031,,-
1019,1038,,-
1019,1039,,-
1019,1040,,-
1019,1041,,-
1019,1042,,-
1019,1044,,-
1019,1063,,-
1019,1065,,-
1019,1068,,-
1019,1074,,-
1019,1076,,-
1019,1078,,-
1019,1080,,-
1019,1082,,-
1019,1083,,-
1019,1089,,-
1019,1090,,-
1019,1106,,-
1019,1115,,-
1019,1121,,-
1019,1128,,-
1019,1131,,-
1019,1147,-,
1020,1022,,-
1020,1028,,-
1020,1030,,-
1020,1032,-,
1020,1037,,-
1020,1040,,-
1020,1041,,-
1020,1068,,-
1020,1080,,-
1020,1100,,-
1020,1130,,-
1021,1029,,-
1021,1030,,-
1021,1031,,-
1021,1032,,-
1021,1039,,-
1021,1040,,-
1021,1049,,-
1021,1078,,-
1021,1079,,-
1021,1080,,-
1021,1083,,-
1021,1106,,-
1021,1120,,-
1021,1121,,-
1022,1038,,-
1022,1040,,-
1022,1041,,-
1022,1042,,-
1022,1044,,-
1022,1049,,-
1022,1077,,-
1022,1089,,-
1022,1090,,-
1022,1095,,-
1022,1098,,-

In [39]:
# G19+G22 Annotations
for item in getting_all_non_overlaps_graph_annotations_which_is_not_in_graph_text:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1018,1102,-,
1018,1128,-,
1018,1134,-,
1018,1155,-,
1019,1049,-,
1020,1059,,-
1020,1066,,-
1020,1088,,-
1020,1090,,-
1021,1063,,
1021,1065,,
1021,1110,,-
1021,1128,-,
1021,1129,,-
1022,1068,,-
1024,1025,,-
1024,1029,,-
1024,1031,,-
1024,1032,,-
1024,1038,,-
1024,1045,,-
1024,1046,,-
1024,1077,,-
1024,1098,,-
1024,1105,,-
1024,1115,,-
1024,1124,,-
1025,1032,,-
1025,1068,,-
1025,1109,,-
1025,1113,,-
1025,1122,,-
1025,1134,,-
1026,1046,,-
1026,1057,,-
1026,1059,,-
1026,1155,,-
1027,1030,,-
1027,1045,,-
1027,1105,,-
1028,1029,,-
1028,1032,,-
1028,1037,,-
1028,1041,,-
1028,1048,,-
1028,1068,,-
1028,1098,,-
1029,1032,,-
1029,1046,,-
1029,1068,,-
1029,1107,,-
1029,1109,,-
1029,1110,,-
1029,1113,,-
1029,1126,,-
1029,1129,,-
1030,1038,,
1030,1045,,
1030,1049,,
1030,1051,,-
1030,1074,,-
1030,1086,,
1030,1092,,-
1030,1093,-,
1030,1108,,-
1030,1113,,-
1030,1122,,-
1030,1126,,-
1030,1129,,-
1030,1155,,-
1031,1032,,-
1031,1068,,
1031,1077,,
1031,1092,-,
1031,1093,,-
1031,1103,,-
1031,1107,,-
1031,11

In [40]:
# G19+G22 Text
for item in all_which_are_in_the_other_and_determined_incorrect_by_graph:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1019,1035,,-
1019,1053,,-
1019,1054,,-
1019,1055,,-
1019,1058,,-
1019,1069,,-
1019,1070,,-
1019,1084,,-
1019,1085,,-
1019,1120,,-
1019,1123,,-
1020,1021,,-
1020,1036,,-
1020,1056,,-
1020,1057,,-
1020,1060,,-
1020,1061,,-
1020,1062,,-
1020,1063,,-
1020,1064,,-
1020,1065,,-
1020,1073,,-
1020,1074,,-
1020,1081,,-
1020,1082,,-
1020,1087,,-
1020,1089,,-
1020,1091,,-
1020,1092,,-
1020,1093,,-
1020,1095,,-
1021,1026,,-
1021,1068,,-
1021,1075,,-
1021,1076,,-
1021,1077,,-
1021,1098,,-
1022,1099,,-
1022,1115,,-
1024,1026,,-
1024,1028,,-
1024,1030,,-
1024,1037,,-
1024,1039,,-
1024,1040,,-
1024,1041,,-
1024,1044,,-
1024,1049,,-
1024,1052,,-
1024,1075,,-
1024,1076,,-
1024,1078,,-
1024,1103,,-
1024,1106,,-
1024,1116,,-
1024,1119,,-
1024,1121,,-
1024,1125,,-
1025,1026,,-
1025,1027,,-
1025,1029,,-
1025,1030,,-
1025,1031,,-
1025,1037,,-
1025,1038,,-
1025,1039,,-
1025,1040,,-
1025,1041,,-
1025,1042,,-
1025,1044,,-
1025,1045,,-
1025,1046,,-
1025,1048,,-
1025,1049,,-
1025,1051,,-
1025,1052,,-
1025,1075,,-

In [41]:
for item in getting_redundancies_which_are_either_text_and_graph_or_annoations_and_graph:
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1021,1063,,-
1021,1065,,-
1030,1038,,-
1030,1039,,-
1030,1040,,-
1030,1041,,-
1030,1045,,-
1030,1049,,-
1031,1049,,-
1036,1056,,-
1036,1057,,-
1036,1059,,-
1036,1060,,-
1036,1062,,-
1036,1063,,-
1036,1064,,-
1036,1073,,-
1036,1074,,-
1036,1081,,-
1036,1082,,-
1036,1087,,-
1036,1088,,-
1036,1090,,-
1036,1091,,-
1036,1092,,-
1036,1093,,-
1036,1095,,-
1038,1039,,-
1038,1040,,-
1038,1041,,-
1038,1045,,-
1038,1049,,-
1039,1040,,-
1039,1041,,-
1039,1042,,-
1039,1045,,-
1039,1049,,-
1040,1045,,-
1040,1049,,-
1041,1042,,-
1041,1045,,-
1041,1049,,-
1042,1044,,-
1042,1045,,-
1042,1049,,-
1044,1045,,-
1055,1058,,-
1056,1064,,-
1056,1090,,-
1059,1088,,-
1060,1062,,-
1060,1063,,-
1060,1065,,-
1060,1066,,-
1060,1074,,-
1060,1081,,-
1060,1082,,-
1060,1088,,-
1060,1089,,-
1060,1090,,-
1060,1091,,-
1060,1092,,-
1060,1093,,-
1061,1062,,-
1061,1065,,-
1061,1087,,-
1061,1088,,-
1061,1090,,-
1061,1093,,-
1062,1063,,-
1062,1065,,-
1062,1066,,-
1062,1074,,-
1062,1081,,-
1062,1090,,-
1062,1093,,-
1062,1095,,-

In [42]:
for item in overlaps_graph_llm_output_text:
    if not (item in overlaps_graph_llm_output_annotations):
        continue
    main_part = "-"
    benefit = "-"
    sp = item.split("-")
    if len(sp) != 3:
        sp = sp[:2] + ["-".join(sp[2:])]
    if sp[2] == RedundancyType.MAIN_PART_REDUNDANCY.value and len(sp) == 3:
        main_part = ""
    elif sp[2] == RedundancyType.BENEFIT_REDUNDANCY.value and len(sp) == 3:
        benefit = ""
    elif sp[2] == RedundancyType.MAIN_PART_BENEFIT_REDUNDANCY.value:
        main_part = ""
        benefit = ""
    else:
        raise ValueError(f"Unknown redundancy type: {sp}")
    print(f"{sp[0]},{sp[1]},{main_part},{benefit}")

1021,1036,,-
1021,1056,,-
1021,1057,,-
1021,1059,,-
1021,1060,,-
1021,1061,,-
1021,1062,,-
1021,1064,,-
1021,1066,,-
1021,1073,,-
1021,1074,,-
1021,1081,,-
1021,1082,,-
1021,1087,,-
1021,1088,,-
1021,1089,,-
1021,1090,,-
1021,1091,,-
1021,1092,,-
1021,1093,,-
1021,1095,,-
1026,1027,,-
1026,1068,,-
1026,1075,,-
1026,1076,,-
1026,1077,,-
1026,1078,,-
1026,1098,,-
1026,1107,,-
1026,1108,,-
1026,1110,,-
1026,1115,,-
1026,1116,,-
1026,1121,,-
1026,1126,,-
1026,1128,,-
1026,1129,,-
1026,1134,,-
1027,1068,,-
1027,1075,,-
1027,1077,,-
1027,1078,,-
1027,1098,,-
1027,1107,,-
1027,1108,,-
1027,1115,,-
1027,1116,,-
1027,1121,,-
1027,1126,,-
1027,1128,,-
1027,1129,,-
1027,1134,,-
1029,1030,,-
1029,1031,,-
1029,1038,,-
1029,1039,,-
1029,1040,,-
1029,1041,,-
1029,1042,,-
1029,1044,,-
1029,1045,,-
1029,1049,,-
1030,1031,,-
1030,1042,,-
1030,1044,,-
1031,1038,,-
1031,1039,,-
1031,1040,,-
1031,1041,,-
1031,1042,,-
1031,1044,,-
1031,1045,,-
1033,1034,,-
1036,1061,,-
1036,1065,,-
1036,1066,,-
1036,1089,,-

### Finding the same detections

In [43]:
gpt_annotations_g19 = gpt_results_annotations["annoations-redundancy-model-gpt-4o-mini-2024-07-18"]["00_g19.json"]
gpt_annotations_g22 = gpt_results_annotations["annoations-redundancy-model-gpt-4o-mini-2024-07-18"]["00_g22.json"]
gpt_text_g19 = gpt_results_text["text-redundancy-model-gpt-4o-mini-2024-07-18"]["00_g19.json"]
gpt_text_g22 = gpt_results_text["text-redundancy-model-gpt-4o-mini-2024-07-18"]["00_g22.json"]

In [44]:
# cached_entries_just_annotations[0]
# cached_entries_just_text[0]
same_main_part: OrderedSet = OrderedSet()
same_benefit: OrderedSet = OrderedSet()
for text_item in gpt_text_g19 + gpt_text_g22:
    for annotation_item in gpt_annotations_g19 + gpt_annotations_g22:
        if ((text_item["relatedStories"][0] == annotation_item["relatedStories"][0] and
            text_item["relatedStories"][1] == annotation_item["relatedStories"][1]) or
            (text_item["relatedStories"][0] == annotation_item["relatedStories"][1] and
            text_item["relatedStories"][1] == annotation_item["relatedStories"][0])):
                if bool(text_item["mainPartRedundancies"]["mainPartRedundancy"]) and (
                    annotation_item["mainPartRedundancies"]["fullRedundancy"] or
                    annotation_item["mainPartRedundancies"]["partialRedundancy"]):
                        same_main_part.add(f"{text_item['relatedStories'][0]}-{text_item['relatedStories'][1]}-{RedundancyType.MAIN_PART_REDUNDANCY.value}")   
                if bool(text_item["benefitRedundancies"]["benefitRedundancy"]) and (
                    annotation_item["benefitRedundancies"]["fullRedundancy"] or
                    annotation_item["benefitRedundancies"]["partialRedundancy"]):
                        same_benefit.add(f"{text_item['relatedStories'][0]}-{text_item['relatedStories'][1]}-{RedundancyType.BENEFIT_REDUNDANCY.value}") 
        

In [45]:
for _ in same_main_part:
    print(_)

1019-1035-mainPart
1019-1053-mainPart
1019-1054-mainPart
1019-1055-mainPart
1019-1058-mainPart
1019-1069-mainPart
1019-1070-mainPart
1019-1084-mainPart
1019-1085-mainPart
1019-1120-mainPart
1019-1123-mainPart
1020-1021-mainPart
1020-1036-mainPart
1020-1056-mainPart
1020-1057-mainPart
1020-1060-mainPart
1020-1061-mainPart
1020-1062-mainPart
1020-1063-mainPart
1020-1064-mainPart
1020-1065-mainPart
1020-1073-mainPart
1020-1074-mainPart
1020-1081-mainPart
1020-1082-mainPart
1020-1087-mainPart
1020-1089-mainPart
1020-1091-mainPart
1020-1092-mainPart
1020-1093-mainPart
1020-1095-mainPart
1021-1026-mainPart
1021-1036-mainPart
1021-1056-mainPart
1021-1057-mainPart
1021-1059-mainPart
1021-1060-mainPart
1021-1061-mainPart
1021-1062-mainPart
1021-1063-mainPart
1021-1064-mainPart
1021-1065-mainPart
1021-1066-mainPart
1021-1068-mainPart
1021-1073-mainPart
1021-1074-mainPart
1021-1075-mainPart
1021-1076-mainPart
1021-1077-mainPart
1021-1081-mainPart
1021-1082-mainPart
1021-1087-mainPart
1021-1088-ma

In [46]:
for _ in same_benefit:
    print(_)

1071-1101-benefit
1091-1092-benefit
1098-1128-benefit
1226-1303-benefit
1229-1259-benefit
1232-1294-benefit
1237-1264-benefit
1239-1254-benefit
1239-1255-benefit
1239-1290-benefit
1242-1243-benefit
1242-1282-benefit
1242-1284-benefit
1242-1286-benefit
1242-1287-benefit
1243-1282-benefit
1243-1284-benefit
1243-1286-benefit
1243-1287-benefit
1245-1303-benefit
1246-1288-benefit
1249-1256-benefit
1254-1255-benefit
1261-1262-benefit
1263-1292-benefit
1268-1269-benefit
1268-1275-benefit
1269-1275-benefit
1270-1271-benefit
1272-1275-benefit
1278-1307-benefit
1280-1288-benefit
1280-1303-benefit
1280-1305-benefit
1281-1303-benefit
1284-1286-benefit
1284-1287-benefit
1284-1290-benefit
1286-1287-benefit
1286-1291-benefit
1287-1289-benefit
1287-1291-benefit
1288-1291-benefit
1289-1291-benefit
1297-1298-benefit
1297-1299-benefit
1298-1299-benefit
1300-1301-benefit
1300-1302-benefit
1301-1302-benefit
1303-1304-benefit
